In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
data = pd.read_csv("../Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [4]:
X = data.drop(columns=['Exited'])
Y = data["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [5]:
neumeric_features = X_train.select_dtypes(include=["float64", "int64"]).columns
non_neumeric_features = X_train.select_dtypes(include=["object"]).columns

neumeric_transformer = StandardScaler()
non_neumeric_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocesor = ColumnTransformer(
    transformers=[
        ("num", neumeric_transformer, neumeric_features),
        ("non_num", non_neumeric_transformer, non_neumeric_features)
    ]
)

X_train_preprocessed = preprocesor.fit_transform(X_train)
X_test_preprocessed = preprocesor.transform(X_test)

X_train_preprocessed = X_train_preprocessed.toarray()
X_test_preprocessed = X_test_preprocessed.toarray()


In [6]:
model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=X_test_preprocessed.shape[1]))

model.add(Dense(units=32, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))

C:\Users\avianage\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [8]:
model.fit(X_train_preprocessed, y_train, epochs=5, batch_size=32, validation_split=0.3)

Epoch 1/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7744 - loss: 0.5319 - val_accuracy: 0.8250 - val_loss: 0.4154
Epoch 2/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8523 - loss: 0.3616 - val_accuracy: 0.8446 - val_loss: 0.3812
Epoch 3/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8910 - loss: 0.2785 - val_accuracy: 0.8379 - val_loss: 0.3920
Epoch 4/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9212 - loss: 0.2146 - val_accuracy: 0.8242 - val_loss: 0.4344
Epoch 5/5
175/175 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9369 - loss: 0.1666 - val_accuracy: 0.8000 - val_loss: 0.4872


In [9]:
loss, accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 933us/step - accuracy: 0.8153 - loss: 0.4691
Test Loss: 0.4612678289413452, Test Accuracy: 0.8195000290870667
